# Attention Bloğu “Adım 2” Geliştirmeleri: Eklenen Parçalar ve Gerekçeleri

Bu not defteri, **yalnızca eklenen/yenilenen bileşenleri** (Channel Attention ve Spatial Attention tarafı) kod parçalarıyla birlikte sunar.  
Her adım için:

- **Amaç**
- **Hedeflenen kazanım**
- **Stabilite ve genelleme etkisi**
- **Eklenen kod parçası**

başlıkları altında akademik açıklama verilmiştir.


## A) Channel Attention (CA) — Eklenen / Güncellenen Kısımlar

### A1) “avg vs max” birleşimini örnek-bazlı (sample-wise) hale getiren **fusion_router**

**Amaç:**  
Sabit (global) bir birleşim katsayısı yerine, her bir örnek için (B bazında) *avg squeeze* ve *max squeeze* sinyallerine göre birleşim ağırlıklarını üretmek.

**Hedef:**  
- Farklı görüntülerde kanal istatistikleri değiştiği için, tek bir global katsayının tüm veri dağılımına uymaması problemine çözüm.
- **Örnek bazlı adaptif kanal vurgusu**.

**Neyi daha iyi hale getirir:**  
- Kanal maskesinin farklı içerik tiplerine göre ayarlanması (kontrast, doku, parlaklık gibi istatistikler).
- Attention’ın “dataset geneli” sabit kalması yerine *instance-aware* hale gelmesi.

**Eklenen kod (CA içinde):**


In [ ]:
# (CA) A1: Sample-wise fusion router
# Input: cat([avg_s, max_s]) -> (B,2C,1,1)
# Output: logits -> softmax -> fusion_w (B,2)

if self.fusion == "softmax":
    self.fusion_router = nn.Sequential(
        nn.Conv2d(2 * channels, fusion_router_hidden, kernel_size=1, bias=True),
        nn.ReLU(inplace=True),
        nn.Conv2d(fusion_router_hidden, 2, kernel_size=1, bias=True),
    )
else:
    self.fusion_router = None


**Forward içinde eklenen birleşim mantığı:**

- Squeeze çıktıları: `avg_s`, `max_s`  
- Router girişi: `cat([avg_s, max_s])`  
- Router çıktısı: örnek bazlı `fusion_w` (B,2)  
- Birleşim: `z = w0*a + w1*m`

Bu yapı, **hangi squeeze türünün daha güvenilir sinyal ürettiğini** örnek bazında öğrenir.


In [ ]:
# (CA) A1: Forward'da sample-wise fusion weights üretimi
avg_s = self.avg_pool(x)  # (B,C,1,1)
max_s = self.max_pool(x)  # (B,C,1,1)

a = self.mlp(avg_s)       # (B,C,1,1)
m = self.mlp(max_s)       # (B,C,1,1)

s_cat = torch.cat([avg_s, max_s], dim=1)   # (B,2C,1,1)
logits = self.fusion_router(s_cat).flatten(1)  # (B,2)
fusion_w = torch.softmax(logits, dim=1)        # (B,2)

z = fusion_w[:, 0].view(-1, 1, 1, 1) * a + fusion_w[:, 1].view(-1, 1, 1, 1) * m


### A2) Temperature scaling’in opsiyonel learnable olması (inverse-softplus parametreleme)

**Amaç:**  
Maskenin çok erken doygunlaşmasını (0/1 saturasyon) azaltmak ve öğrenme sürecinde “logit ölçeğini” daha kontrollü hale getirmek.

**Hedef:**  
- Temperature sabitken bazı ağlarda gating hızlı saturate olabilir.  
- Learnable temperature ile model, dikkat logitlerinin ölçeğini veriyle uyumlu şekilde ayarlayabilir.

**Neyi daha iyi hale getirir:**  
- Eğitim stabilitesi (özellikle yüksek öğrenme oranı, güçlü regularization veya karma mimarilerde).
- Attention maskesinin “tam kapalı / tam açık” uçlara yığılmasını azaltma.

**Eklenen kod (parametreleme ve get_T):**


In [ ]:
# (CA) A2: Learnable temperature (inverse-softplus parametrization)

def _softplus_inverse(y: torch.Tensor, eps: float = 1e-6) -> torch.Tensor:
    return torch.log(torch.clamp(torch.exp(y) - 1.0, min=eps))

self.learnable_temperature = bool(learnable_temperature)
if self.learnable_temperature:
    t0 = torch.tensor(float(temperature))
    t_inv = _softplus_inverse(t0, eps=self.eps)
    self.t_raw = nn.Parameter(t_inv)
else:
    self.register_buffer("T", torch.tensor(float(temperature)))

def get_T(self) -> torch.Tensor:
    if self.learnable_temperature:
        return F.softplus(self.t_raw) + self.eps
    return self.T


### A3) Debug çıktısı: `fusion_w` (B,2) döndürme

**Amaç:**  
Eğitim/ablation sırasında router’ın “avg vs max” tercihini izlemek; router’ın tek bir kola çökmesi veya anlamsız dağılım üretmesi gibi durumları erken tespit etmek.

**Hedef:**  
- Analiz edilebilirlik (interpretability)
- Deney tekrarlanabilirliği (ablation raporlama)

**Eklenen kod (CA forward çıktısı):**


In [ ]:
# (CA) A3: Debug return
if self.return_fusion_weights and (fusion_w is not None):
    return y, ca, fusion_w
return y, ca


## B) Spatial Attention (SA) — Eklenen / Güncellenen Kısımlar

### B1) Spatial girişe Coordinate (CoordConv) kanalları ekleme

**Amaç:**  
Salt `avg_map` ve `max_map` ile üretilen spatial ipuçları, konumsal referansı (x/y koordinatları) *doğrudan* taşımaz. CoordConv kanalları, modele “nerede?” bilgisini düşük maliyetle sağlar.

**Hedef:**  
- Konum farkındalığı (position awareness)
- Simetrik/benzer örüntülerde (ör. aynı doku farklı konum) ayrım gücünü artırmak

**Neyi daha iyi hale getirir:**  
- Spatial maskenin “konuma duyarlı” hale gelmesi
- Multi-scale branch’lerin ürettiği maskelerin daha anlamlı bölgelere odaklanması

**Eklenen kod (SA forward içinde coord grid):**


In [ ]:
# (SA) B1: Coord grid üretimi ve input'a ekleme
avg_map = torch.mean(x, dim=1, keepdim=True)          # (B,1,H,W)
max_map, _ = torch.max(x, dim=1, keepdim=True)        # (B,1,H,W)

# normalize: [-1,1] (alternatif olarak [0,1])
xs = torch.linspace(-1.0, 1.0, W, device=x.device, dtype=x.dtype)
ys = torch.linspace(-1.0, 1.0, H, device=x.device, dtype=x.dtype)
yy, xx = torch.meshgrid(ys, xs, indexing="ij")

x_coord = xx.unsqueeze(0).unsqueeze(0).expand(B, -1, -1, -1)  # (B,1,H,W)
y_coord = yy.unsqueeze(0).unsqueeze(0).expand(B, -1, -1, -1)  # (B,1,H,W)

s = torch.cat([avg_map, max_map, x_coord, y_coord], dim=1)     # (B,4,H,W)


### B2) Branch konvolüsyonlarını Depthwise + Pointwise yapma

**Amaç:**  
Her branch’in parametre/maliyetini azaltırken, çok ölçekli (multi-scale) maskeyi korumak.

**Hedef:**  
- Aynı sayıda branch ile daha düşük parametre ve MAC
- Ölçek çeşitliliğini artırırken hesap maliyetini kontrol etmek

**Neyi daha iyi hale getirir:**  
- Özellikle yüksek çözünürlükte veya çok kernel’li havuzlarda verimlilik
- Branch’lerin birbirini kopyalaması yerine daha “ince” (lightweight) uzmanlaşma

**Eklenen kod (branch modülü):**


In [ ]:
# (SA) B2: Depthwise + Pointwise branch
class _DWPointwiseBranch(nn.Module):
    def __init__(self, in_ch: int, k: int, dilation: int = 1):
        super().__init__()
        k = (k if k % 2 == 1 else k + 1)
        pad = dilation * (k - 1) // 2

        self.dw = nn.Conv2d(
            in_ch, in_ch,
            kernel_size=k,
            padding=pad,
            dilation=dilation,
            groups=in_ch,
            bias=False,
        )
        self.pw = nn.Conv2d(in_ch, 1, kernel_size=1, bias=False)

    def forward(self, s: torch.Tensor) -> torch.Tensor:
        return self.pw(self.dw(s))


### B3) Router’ın CoordConv’lu input’a göre güncellenmesi (4 kanal)

**Amaç:**  
Router’ın branch ağırlıklarını, yalnızca (avg,max) değil, aynı zamanda (x,y) konumsal ipuçlarıyla belirlemesi.

**Hedef:**  
- Router kararlarının içerik + konum bileşimine göre verilmesi
- Farklı bölgelerde farklı ölçekte kernel tercihinin öğrenilmesi

**Neyi daha iyi hale getirir:**  
- Branch ağırlıklarının anlamsız uniform dağılması veya tek kola çökmesi riskini azaltma
- Çok ölçekli maskenin örnek bazında daha seçici kullanılması

**Eklenen kod (router):**


In [ ]:
# (SA) B3: Router (in_ch=4) -> hidden -> K
self.router = nn.Sequential(
    nn.AdaptiveAvgPool2d(1),
    nn.Conv2d(4, router_hidden, kernel_size=1, bias=bias),
    nn.ReLU(inplace=True),
    nn.Conv2d(router_hidden, self.num_branches, kernel_size=1, bias=bias),
)

# forward:
logits = self.router(s).flatten(1)     # (B,K)
rw = torch.softmax(logits, dim=1)      # (B,K)


### B4) “Kontrollü dinamik” branch havuzu (kernels + optional dilated)

**Amaç:**  
Kernel setini parametrik tutarak (3,5,7,9,...) çok ölçekli maske üretimini genişletebilmek; gerekirse dilated bir branch ile daha geniş bağlam eklemek.

**Hedef:**  
- Multi-scale kapasiteyi kolay ölçeklenebilir yapmak
- Dilated branch ile daha geniş receptive field (özellikle büyük nesne/struktur)

**Neyi daha iyi hale getirir:**  
- Spatial maskenin farklı ölçekte ipuçlarını yakalaması
- Veri setine göre kernel havuzunun kolay yeniden konfigüre edilmesi

**Eklenen kod (branch havuzu kurulumu):**


In [ ]:
# (SA) B4: Branch pool
branches = []
for k in kernels:
    k = (k if k % 2 == 1 else k + 1)
    branches.append(_DWPointwiseBranch(in_ch=4, k=k, dilation=1))

if use_dilated:
    dk = (dilated_kernel if dilated_kernel % 2 == 1 else dilated_kernel + 1)
    branches.append(_DWPointwiseBranch(in_ch=4, k=dk, dilation=dilated_d))

self.branches = nn.ModuleList(branches)
self.num_branches = len(self.branches)


### B5) Maske üretimi: branch stack + router ağırlıklı toplama + temperature + gate

**Amaç:**  
Branch çıktılarının (B,K,1,H,W) şeklinde üretilip router ağırlıklarıyla örnek bazında birleştirilmesi; ardından temperature ve gate ile maske üretimi.

**Hedef:**  
- Örnek bazlı multi-scale maske
- Maskenin doygunluğunu temperature ile kontrol etmek

**Neyi daha iyi hale getirir:**  
- Router’ın “hangi ölçekte” çalışacağını örnek bazında seçmesi
- Maskenin daha stabil ve daha anlamlı öğrenilmesi

**Eklenen kod (forward maske üretimi):**


In [ ]:
# (SA) B5: Weighted branch aggregation + gating
z = torch.stack([br(s) for br in self.branches], dim=1)    # (B,K,1,H,W)
wlogit = (rw[:, :, None, None, None] * z).sum(dim=1)       # (B,1,H,W)

T = self.get_T()
sa = self.gate_fn(wlogit / T)
y = x * sa

# debug:
if self.return_router_weights:
    return y, sa, rw
return y, sa


## Kısa değerlendirme: Stabilite ve izlenebilirlik

- **Örnek bazlı router (CA/SA):** Modelin “tek bir global katsayıya” kilitlenmesini engeller; farklı örneklerde farklı attention davranışı üretir.  
- **Temperature ölçekleme:** Maskelerin erken saturasyonunu azaltarak eğitim stabilitesini artırır.  
- **CoordConv (SA):** Konumsal referans ekleyerek maskenin mekânsal ayrım gücünü artırır.  
- **Depthwise+Pointwise branch (SA):** Çok ölçekli kapasiteyi korurken maliyeti düşürür; kernel havuzu büyütmeyi pratik hale getirir.  
- **Debug çıktıları (fusion_w / router_w):** Router çökmesi, uniform dağılma veya beklenmeyen davranışlar deney sırasında nicel olarak izlenebilir.
